In [1]:
import sys
import os
sys.path.append(os.path.abspath('..'))  # or the full path to the "project" directory
import numpy as np
import gpbr

In [2]:
from gpbr.direct.heat_equation.common import MFSConfig, Dimension

T = 1 # final time
N = 30
m1 = 16
m2 = 8
M = m1 * m2 # number of collocation points
ETA1 = 0.5
ETA2 = 2.0
config = MFSConfig(
    N=N,
    n_coll=M,
    n_source=M,
    T=T,
    eta1=ETA1,
    eta2=ETA2,
    f1=None,
    f2=None,
    dim = Dimension.THREE_D,
    n_coll_theta=m1,
    n_coll_phi=m2,
    n_source_theta=m1,
    n_source_phi=m2
)


In [3]:
from gpbr.direct.heat_equation.helpers import precalculate_mfs_data
mfs_data = precalculate_mfs_data(config)

In [4]:
from gpbr.direct.heat_equation.polynomial import calculate_3d_polinomials
polynomials = calculate_3d_polinomials(mfs_data.N, mfs_data.nu, mfs_data.Beta)


In [5]:
with np.printoptions(precision=4, suppress=True):
    print(polynomials.A)

[[       1.               nan           nan           nan           nan
            nan           nan           nan           nan           nan
            nan           nan           nan           nan           nan
            nan           nan           nan           nan           nan
            nan           nan           nan           nan           nan
            nan           nan           nan           nan           nan
            nan]
 [       1.            7.874            nan           nan           nan
            nan           nan           nan           nan           nan
            nan           nan           nan           nan           nan
            nan           nan           nan           nan           nan
            nan           nan           nan           nan           nan
            nan           nan           nan           nan           nan
            nan]
 [       1.            3.937        31.               nan           nan
            nan           nan 

In [6]:
for p in polynomials.polynomials:
    print(p)

1.0
1.0 + 7.87400787·x
1.0 + 3.93700394·x + 31.0·x²
1.0 + 7.87400787·x + 0.0·x² + 81.36474803·x³
1.0 + 4.92125492·x + 38.75·x² - 40.68237402·x³ + 160.16666667·x⁴
1.0 + 7.87400787·x + 0.0·x² + 142.38830906·x³ - 160.16666667·x⁴ +
252.2307189·x⁵
1.0 + 5.41338041·x + 42.625·x² - 81.36474803·x³ + 400.41666667·x⁴ -
378.34607835·x⁵ + 331.01111111·x⁶
1.0 + 7.87400787·x + (3.60956071e-15)·x² + 193.24127657·x³ -
400.41666667·x⁴ + 882.80751614·x⁵ - 662.02222222·x⁶ + 372.34058504·x⁷
1.0 + 5.72095885·x + 45.046875·x² - 119.50447367·x³ + 690.71875·x⁴ -
1166.5670749·x⁵ + 1572.30277778·x⁶ - 930.8514626·x⁷ + 366.4765873·x⁸
1.0 + 7.87400787·x + (3.60956071e-15)·x² + 237.73762315·x³ -
690.71875·x⁴ + 1907.49481166·x⁵ - 2482.58333333·x⁶ + 2327.12865649·x⁷ -
1099.4297619·x⁸ + 320.62661489·x⁹
1.0 + 5.93626375·x + 46.7421875·x² - 155.10155093·x³ + 1016.05729167·x⁴ -
2356.7807797·x⁵ + 4158.32708333·x⁶ - 4188.83158169·x⁷ + 2931.81269841·x⁸ -
1122.19315213·x⁹ + 252.46164903·x¹⁰
1.0 + 7.87400787·x + (3.24860464e-

In [15]:
points = np.linspace(-1, 1, 500)
points

array([-1.        , -0.99599198, -0.99198397, -0.98797595, -0.98396794,
       -0.97995992, -0.9759519 , -0.97194389, -0.96793587, -0.96392786,
       -0.95991984, -0.95591182, -0.95190381, -0.94789579, -0.94388778,
       -0.93987976, -0.93587174, -0.93186373, -0.92785571, -0.9238477 ,
       -0.91983968, -0.91583166, -0.91182365, -0.90781563, -0.90380762,
       -0.8997996 , -0.89579158, -0.89178357, -0.88777555, -0.88376754,
       -0.87975952, -0.8757515 , -0.87174349, -0.86773547, -0.86372745,
       -0.85971944, -0.85571142, -0.85170341, -0.84769539, -0.84368737,
       -0.83967936, -0.83567134, -0.83166333, -0.82765531, -0.82364729,
       -0.81963928, -0.81563126, -0.81162325, -0.80761523, -0.80360721,
       -0.7995992 , -0.79559118, -0.79158317, -0.78757515, -0.78356713,
       -0.77955912, -0.7755511 , -0.77154309, -0.76753507, -0.76352705,
       -0.75951904, -0.75551102, -0.75150301, -0.74749499, -0.74348697,
       -0.73947896, -0.73547094, -0.73146293, -0.72745491, -0.72

In [16]:
for n, polinomial in enumerate(polynomials.polynomials):
    v = polinomial(points)
    dv = polinomial.deriv()(points)
    d2v = polinomial.deriv(2)(points)
    right = 0
    for m in range(n):
        right += mfs_data.Beta[n-m]*polynomials.polynomials[m](points)
    delta = d2v - 2*mfs_data.nu*dv - right
    print(f"n={n}")
    # print(max(abs(delta)))
    print(np.linalg.norm(delta))
    # print(max(d2v - 2*mfs_data.nu*dv - right))

n=0
0.0
n=1
0.0
n=2
1.3464925450640537e-12
n=3
5.627642009376331e-12
n=4
1.742717417713773e-11
n=5
4.769156067277664e-11
n=6
1.2630830992607026e-10
n=7
2.343548377197457e-10
n=8
5.929808219999437e-10
n=9
7.789427774780795e-10
n=10
2.422902626388206e-09
n=11
2.8514504873278373e-09
n=12
4.826925717387295e-09
n=13
1.0088786850648102e-08
n=14
1.801729795688257e-08
n=15
1.8994163156467936e-08
n=16
5.126429817971597e-08
n=17
6.196081885622697e-08
n=18
1.3389481923035085e-07
n=19
1.787787563253904e-07
n=20
2.2138911389737022e-07
n=21
4.37855413890654e-07
n=22
5.14079876959521e-07
n=23
9.080223177655201e-07
n=24
1.0345485864566857e-06
n=25
2.0236744695153855e-06
n=26
2.214684647001449e-06
n=27
4.659044664680779e-06
n=28
4.5013258648291735e-06
n=29
1.0593806848896713e-05
n=30
1.1102295531872372e-05


In [9]:
assert False

AssertionError: 

In [ ]:
beta = np.ones(N+1)
nu = beta[0]
mfs_data = MfSData(beta,nu)

In [ ]:
mfs_data

MfSData(Beta=array([1., 1., 1., 1., 1., 1., 1., 1., 1.]), nu=1.0)

In [ ]:
polinomials_data = calculate_3d_polinomials(mfs_data, N)

In [ ]:
with np.printoptions(precision=4, suppress=True):
    print(polinomials_data.A)

[[ 1.         nan     nan     nan     nan     nan     nan     nan     nan]
 [ 1.     -0.5        nan     nan     nan     nan     nan     nan     nan]
 [ 1.     -0.875   0.125      nan     nan     nan     nan     nan     nan]
 [ 1.     -1.1875  0.3125 -0.0208     nan     nan     nan     nan     nan]
 [ 1.     -1.4609  0.5391 -0.0677  0.0026     nan     nan     nan     nan]
 [ 1.     -1.707   0.793  -0.1419  0.0104 -0.0003     nan     nan     nan]
 [ 1.     -1.9326  1.0674 -0.2435  0.0257 -0.0012  0.         nan     nan]
 [ 1.     -2.1421  1.3579 -0.3719  0.0505 -0.0035  0.0001 -0.         nan]
 [ 1.     -2.3385  1.6615 -0.5265  0.0863 -0.0078  0.0004 -0.      0.    ]]


In [ ]:
for p in polinomials_data.polinomials:
    print(p)

1.0
1.0 - 0.5 x
1.0 - 0.875 x + 0.125 x**2
1.0 - 1.1875 x + 0.3125 x**2 - 0.02083333 x**3
1.0 - 1.4609375 x + 0.5390625 x**2 - 0.06770833 x**3 + 0.00260417 x**4
1.0 - 1.70703125 x + 0.79296875 x**2 - 0.14192708 x**3 + 0.01041667 x**4 -
0.00026042 x**5
1.0 - 1.93261719 x + 1.06738281 x**2 - 0.24348958 x**3 + 0.02571615 x**4 -
0.00123698 x**5 + (2.17013889e-05) x**6
1.0 - 2.14208984 x + 1.35791016 x**2 - 0.37190755 x**3 + 0.05045573 x**4 -
0.00351563 x**5 + 0.00011936 x**6 - (1.55009921e-06) x**7
1.0 - 2.33847046 x + 1.66152954 x**2 - 0.5265096 x**3 + 0.08631388 x**4 -
0.00776774 x**5 + 0.00038384 x**6 - (9.68812004e-06) x**7 +
(9.68812004e-08) x**8


In [ ]:
points = np.arange(0, 20)
points

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])

In [ ]:
for n, polinomial in enumerate(polinomials_data.polinomials):
    v = polinomial(points)
    dv = polinomial.deriv()(points)
    d2v = polinomial.deriv(2)(points)
    right = 0
    for m in range(n):
        right += mfs_data.Beta[n-m]*polinomials_data.polinomials[m](points)
    print(max(d2v - 2*mfs_data.nu*dv - right))

0.0
0.0
0.0
0.0
3.552713678800501e-15
1.3322676295501878e-14
5.3290705182007514e-14
1.1546319456101628e-14
1.3482548411047901e-12
